In [34]:
import tensorflow as tf
import numpy as np

NUM_FEATURES = 1
NUM_CLASSES = 2 # Output is binary, just the probability of liking that meal.
NUM_TRAIN = 1000
BATCH_SIZE = 4

def CreateModel():
  """
  Returns training and accuracy op graphs.
  """
  x = tf.placeholder(tf.float32, [None, NUM_FEATURES])
  W = tf.Variable(tf.zeros([NUM_FEATURES, NUM_CLASSES]))
  b = tf.Variable(tf.zeros([NUM_CLASSES]))
  y = tf.matmul(x, W) + b
  
  y_ = tf.placeholder(tf.int64, [None])
  cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y)
  train_step = tf.train.GradientDescentOptimizer(.5).minimize(cross_entropy)
  
  correct_prediction = tf.equal(tf.argmax(y, 1), y_)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
  return train_step, accuracy, x, y_

def TestModel(model, input_matrix):
  """
  Takes model, input_matrix and prints accuracy information.
  
  input_matrix is of size (None, NUM_FEATURES + 1).
  It contains meal history of one user.
  """
  train_step, accuracy, x, y_ = model
  
  sess = tf.InteractiveSession()
  tf.global_variables_initializer().run()
  
  training_size = int(input_matrix.shape[0]*.8)
  
  for _ in range(NUM_TRAIN):
    rand_indices = np.random.choice(training_size, BATCH_SIZE)
    batch_xs, batch_ys = np.hsplit(input_matrix[rand_indices], [-1])
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys.flatten()})
  
  test_xs, test_ys = np.hsplit(input_matrix[training_size:], [-1])
  print(sess.run(accuracy, feed_dict={x:test_xs, y_:test_ys.flatten()}))

In [35]:
fields = {"user_id":0, "suggestee_id":1, "latitude":2, "longitude":3, "timestamp":4}
field_ids = [0]*len(fields)

inputs_per_user = {}
with open("dataset.csv", "r") as raw_data_file:
  headers = raw_data_file.readline().split(',')
  for idx, header in enumerate(headers):
    if header in fields:
      field_ids[fields[header]] = idx
  
  for raw_line in raw_data_file:
    line = np.array(raw_line.split(','))
    line = line[field_ids].astype(np.float)
    user_id = int(line[0])
    if user_id not in inputs_per_user:
      inputs_per_user[user_id] = np.ndarray([0, NUM_FEATURES + 1])
    features = [line[1:1+NUM_FEATURES], 1]
    inputs_per_user[user_id] = np.vstack((inputs_per_user[user_id], features))
  
  model = CreateModel()
  for user_id in inputs_per_user:
    input_matrix = inputs_per_user[user_id]
    if input_matrix.shape[0]>2:
      print("Info for user_id:", user_id)
      TestModel(model, input_matrix)

Info for user_id: 2
1.0
Info for user_id: 9
1.0
Info for user_id: 1
1.0
